# Прикладная статистика. ДЗ 1.
# Академия Аналитиков Авито

__Правила:__
- Финальный дедлайн: **2022-11-30 23:59**. 
- После того как ваше решение проверят и ответят, вам дается неделя на исправление тех задач, о которых скажет проверяющий. Ответ и обсуждение решения — в телеграме.

- Выполненную работу нужно отправить
    - в чатик HW1-<ваше имя> через бота @AAA_stats23_bot
    - или в личные сообщения боту.
- В качестве решения нужно отправить файл ipynb. Ссылка на интернет-ресурсы не принимается. Не публикуйте решения в открытом доступе!
- Для выполнения задания используйте этот ноутбук в качествие основы, ничего не удаляя из него. **При этом можно добавлять новые ячейки!**
- в ячейках с комменарием `#Автопроверка` нужно заполнить содержимое функций и классов (если есть), которые будут уже объявлены в этой ячейке. При этом:
    - Нельзя убрирать или переставять `#Автопроверка` в ячейке. 
    - Нельзя менять сигнатуру и возвращаемое значение функций. То есть добавлять любой код можно, но удалять, что уже написано - нельзя.
    - Нельзя ничего импортировать в таких ячейках. Все доступные для использования библиотеки будут указаны заранее. Такие слова, как `import`, `globals`, `locals`, `eval`, `exec` также нельзя использовать внутри ячеек.
    - Нельзя использовать библиотеки, кроме тех, что указаны в задании. Ваш код должен работать именно с эти набором библиотек без любого дополнительного импорта!
    - Нельзя использовать код из других ячеек ноутбука (кроме ячейки с импортом, в которой указаны все доступные библиотеки). Единственное исключение - если вы проставите в начало такой ячейки слово `#Автопроверка`. Тогда вы можете использовать код из этой ячейки.
    - В случае нарушения этого правила автопроверка будет провалена и вы не получите часть баллов за задачу. 
    - В случае, если есть несколько ячеек автопроверки, то в каждой такой ячейке можно использовать созданные вами функции (или классы) из других ячеек автопроверки.

In [39]:
from collections import namedtuple
from scipy.stats import binom
import math
import numpy as np

# Общие замечания по задачам с автопроверкой

Во всех задачах далее, где потребуется автопроверка, можно использовать только следующие библиотеки:

```
from collections import namedtuple
from scipy.stats import binom
import math
import numpy as np
```

Еще раз обращаем внимание, что в ячейках автопроверки __не__ должны быть импортированы какие-либо библиотеки. В других ячейках импортировать библиотеки можно, но при проверке использованы будут только указанные.

## Задача 1. 3 балла

[Осьминог Пауль](https://www.championat.com/football/article-3238881-samye-izvestnye-zhivotnye-predskazateli.html) 14 раз пробовал угадать победителя футбольного матча.
12 раз он угадал правильно, 2 раза — ошибся. Мы хотим проверить гипотезу:

 - $H_0$: осьминог угадывает победителя матча с вероятностью 0.5
 - $H_1$: осьминог выбирает победителя  матча с вероятностью $\neq$ 0.5

#### Пункт А. 1 балл: 

### На зачет

Вам нужно разработать статистический критерий для проверки этой гипотезы, а также посчитать p-value.

$H_0: p = 0.5$  
$H_1: p \neq 0.5$  
$S(X) = \sum\limits_{i=1}^n X_i \sim Binom(n, 0.5)$ при верности гипотезы $H_0$  
Критерий: $\bigl\{ S(X) \gt u_{1-\frac{\alpha}{2}} \bigr\} \cup \bigl\{ S(X) \lt u_{\frac{\alpha}{2}} \bigr\}$, где $u_{\beta}$ - критическое значение, равное $\beta$-квантилю $Binom(n, 0.5)$,  
Pvalue = $2 \cdot \text{min}\bigl[P\bigl( S(X) \geqslant S_0 \, | \, H_0\bigr), \, P\bigl( S(X) \leqslant S_0 \, | \, H_0\bigr) \bigr]$, где $S_0$ - значение статистики от заданной выборки 

Реализуйте критерий `check_paul_criterion(n, k, alpha)`, где 
- `n` &mdash; количество матчей;
- `k` &mdash; количество верных предсказаний от осьминога;
- `alpha` &mdash; уровень значимости критерия.

Функция должна вернуть `PaulCheckResults` с полями
- is_rejected: отверглась или нет гипотеза H_0 на уровне значимости alpha
- pvalue


In [40]:
# Автопроверка

PaulCheckResults = namedtuple('PaulCheckResults', ['is_rejected', 'pvalue'])

def check_paul_criterion(n: int, k: int, alpha: float = 0.05):
    """
    Параметры:
    - n: количество матчей
    - k: количество верных предсказаний от осьминога
    - alpha: уровень значимости критерия.
        
    Возвращает:
    - PaulCheckResults с полями:
        - is_rejected: bool
            - отверглась или нет гипотеза H_0 
                       на уровне значимости alpha
        - pvalue: float
    """

    is_rejected = None
    pvalue = None
    p0 = 0.5

    right_pvalue = 1 - binom(n, p0).cdf(k)
    left_pvalue = binom(n, p0).cdf(k)
    pvalue = 2 * min(left_pvalue, right_pvalue)
    is_rejected = pvalue < alpha

    return PaulCheckResults(is_rejected, pvalue)


#### Пункт B. 2 балла: 

Какие выводы можно сделать из полученного результата?

**Определение критической области кр-ия для заданного n**

In [41]:
n = 14
k = 12
alpha = 0.05

print(check_paul_criterion(n, k, alpha))
low_critical_value = binom.ppf(alpha/2, n, 0.5)
high_critical_value = binom.ppf(1 - alpha/2, n, 0.5)
print(f"S(X) > {high_critical_value} or S(X) < {low_critical_value} ==> Reject H_0")

PaulCheckResults(is_rejected=True, pvalue=0.0018310546875)
S(X) > 11.0 or S(X) < 3.0 ==> Reject H_0


**Вывод:**  
1. На уровне значимости в 5% гипотеза о случайности рез-тов осьминога отвергается. (Этот осьминог всё же что-то знал...)  
2. Критическая область для данных гипотезы, статистики и уровня значимости имеет вид: $\bigl \{ S(X) \geqslant 11 \bigr \}$

**Проверка корректности найденных критических значений**

In [42]:
print(str(binom.cdf(2, n, 0.5) + 1 - binom.cdf(11, n, 0.5)) + f" < {alpha}")
print(str(binom.cdf(3, n, 0.5) + 1 - binom.cdf(10, n, 0.5)) + f" > {alpha}")

0.012939453125 < 0.05
0.057373046875 > 0.05


**Вывод:**  
Фактический уровень значимости для найденных критических значений меньше, чем альфа.  
В то время как для изменённых на единицу значений фактический уровень значимости больше, чем альфа.  
Значит, полученные критические значения корректны с точки зрения лимита на ошибку 1го рода

**Построение доверительного интервала**  
Здесь я использую следующее определение доверительного интервала:  
> Пусть есть статистика $Q$ и критерий $\psi(Q)$ для проверки гипотезы $H_0: \theta = m$ уровня значимости $\alpha$.
>
> Тогда доверительный интервал для $\theta$ уровня доверия $1 - \alpha$: множество таких m, что критерий $\psi(Q)$ не отвергает для них $H_0$.

In [34]:
def check_criterion(n: int, k: int, alpha: float = 0.05, p0: float = 0.5):
    """
    Параметры:
    - n: количество матчей
    - k: количество верных предсказаний от осьминога
    - alpha: уровень значимости критерия
    - p0: вероятность в нулевой гипотезе
        
    Возвращает:
    - is_rejected: bool
        - отверглась или нет гипотеза H_0 
                   на уровне значимости alpha
    """
    right_pvalue = 1 - binom(n, p0).cdf(k-1)
    left_pvalue = binom(n, p0).cdf(k)
    pvalue = 2 * min(left_pvalue, right_pvalue)
    is_rejected = pvalue < alpha

    return is_rejected


p_arr = np.linspace(0, 1, 1001)
left_border, right_border = None, None

for p0 in p_arr:
    is_rejected = check_criterion(n, k, alpha, p0)
    
    if is_rejected == False:
        if left_border is None:
            left_border = p0
            
        right_border = p0
        
print(f"CI for p = ({round(left_border, 3)}, {round(right_border, 3)})")

CI for p = (0.572, 0.982)


**Вывод:**  
Поскольку 0.5 не входит в 95% ДИ параметра $p$ (ввиду вычисленного ранее p-value 0.5 не войдёт и в 98.5% ДИ), то можно с уверенностью заявить, что осьминог является оракулом

## Задача 2. 3 балла

### На зачет

Мы разработали новый дизайн нашего продукта. Вероятность, что он понравится случайному человеку — $p$, и она нам неизвестна. Мы хотим
проверить $H_0: p = 1$ с помощью статистического критерия c уровнем значимости $\alpha$. Предложить критерий для решения этой задачи.

$H_0: p = 1$  
$H_1: p < 1$  
$S(X) = \sum\limits_{i=1}^n X_i \sim Binom(n, 1)$ при верности гипотезы $H_0$  
Критерий: $\bigl\{ S(X) \leqslant u_{\alpha} \bigr\}$, где $u_{\alpha}$ - критическое значение, равное $\alpha$-квантилю $Binom(n, 1)$, уменьшенному на 1  
Pvalue = $P\bigl( S(X) \leqslant S_0 \, | \, H_0\bigr)$, где $S_0$ - значение статистики от заданной выборки 

Напишите функцию `calculate_number_of_users(alpha, beta, p)` — скольки людям нужно показать этот дизайн, чтобы добиться мощности `1 - beta` при заданном `p` и уровне значимости `alpha`.

P.S. Утверждается, что на наших тестах ответ не будет превосходить 1000 человек.

Данная функция пробегается в цикле по $n$ - размеру выборки и проверяет, что мощность кр-ия при истинном значении параметра $p$ больше $1 - \beta$.  
В силу нулевой гипотезы критическое значение будет равно $n - 1$ вне зависимости от $\alpha$

In [7]:
#Автопроверка

def calculate_number_of_users(alpha: float, beta: float, p: float):
    """
    Параметры:
    - alpha: уровень значимости
    - beta: инвертированная мощность критерия. мощность = 1 - beta.
    - p: истинная вероятность того, что пользователю понравится дизайн.
    Возвращает:
    - number_of_users: int
        - количество людей, которым надо показать дизайн.
    """
    number_of_users = None
    N_MAX = 1000
    
    for n in range(2, N_MAX+1):
        critical_value = n - 1
        power = binom.cdf(critical_value, n, p)
        
        if power >= 1 - beta:
            number_of_users = n
            break
    else:
        number_of_usres = N_MAX
        
    return number_of_users

## Задача 3.

По недостоверной информации (вероятность, что она верна, считаем за 1%), в новой версии нашего сайта есть сложнодетектируемый баг. Мы могли бы попросить разработку его отыскать и починить, но на это уйдет много ресурсов.

К счастью, у нас есть старый AB тест (новая версия сайта vs старая), который мы можем проанализировать и с некоторой вероятностью обнаружить наличие бага просто сравнением выборок. У нас есть три критерия для проверки гипотезы "$H_0$: баги нет, $H_1$: баг есть":
- критерий `A`: $\alpha = 0.02, 1-\beta = 0.50$
- критерий `B`: $\alpha = 0.05, 1-\beta = 0.60$
- критерий `C`: $\alpha = 0.10, 1-\beta = 0.70$

Если критерий находит баг, мы просим разработчиков потратить силы и починить. На это у них уйдет усилий на 1 М ₽ независимо от того, найдут они баг или нет.
Если критерий не найдет баг, затраты разработчиков будут нулевыми, но из-за бага мы потеряем в конечном итоге 50 М ₽.

#### Пункт А. 2 балла: 

Какой критерий стоит выбрать?

<u>Введём обозначения</u>:

$S_1$ - затраты на поиск и исправление бага  
$S_2$ - издержки бага  
$p_{bug}$ - вероятность наличия бага

<u>Интерпретация ошибок 1го и 2го рода</u>:

$\alpha = P\bigl( \text{сказали, что баг есть} | \text{бага нет} \bigr)$  
$\beta = P\bigl( \text{сказали, что бага нету} | \text{баг есть} \bigr)$

**Затраты критерия:**  
$S_{cr} = p_{bug} \cdot \bigl[ \beta S_2 + (1 - \beta) S_1 \bigr] + (1 - p_{bug}) \cdot \bigl[ \alpha S_1 + (1 - \alpha) \cdot 0\bigr]$

In [8]:
def criterion_cost(alpha, beta, p_bug, s1, s2):
    first_summand = p_bug * (beta * s2 + (1 - beta) * s1)
    second_summand = (1 - p_bug) * (alpha * s1)
    
    return first_summand + second_summand


s1 = 1
s2 = 50
p_bug = 0.01

cr_a_cost = criterion_cost(0.02, 1-0.5, p_bug, s1, s2)
cr_b_cost = criterion_cost(0.05, 1-0.6, p_bug, s1, s2)
cr_c_cost = criterion_cost(0.10, 1-0.7, p_bug, s1, s2)

print(f"Стоимость критерия A равна {round(cr_a_cost, 3)} M ₽")
print(f"Стоимость критерия B равна {round(cr_b_cost, 4)} M ₽")
print(f"Стоимость критерия C равна {round(cr_c_cost, 4)} M ₽")

Стоимость критерия A равна 0.275 M ₽
Стоимость критерия B равна 0.2555 M ₽
Стоимость критерия C равна 0.256 M ₽


**Ответ:** Следует выбрать критерий B

#### Пункт B. 2 балла: 
Предложите оптимальную стратегию, если потери от ненайденного бага составят вместо 50М:
- 20М рублей;
- 3М рублей;
- 300М рублей.

### 1. Стратегия = кр-ий с минимальными затратами

In [9]:
s2_arr = [20, 3, 300]
cr_names = ['A', 'B', 'C']

for s2 in s2_arr:
    cr_a_cost = criterion_cost(0.02, 1-0.5, p_bug, s1, s2)
    cr_b_cost = criterion_cost(0.05, 1-0.6, p_bug, s1, s2)
    cr_c_cost = criterion_cost(0.10, 1-0.7, p_bug, s1, s2)
    
    cost_arr = (cr_a_cost, cr_b_cost, cr_c_cost)
    best_criterion_ind = np.argmin(cost_arr)
    best_cr_name = cr_names[best_criterion_ind]
    best_cr_cost = cost_arr[best_criterion_ind]
    
    print(f"Для стоимости бага, равной {s2} M ₽, стоит выбрать кр-ий {best_cr_name}")
    print(f"Его стоимость составляет {round(best_cr_cost, 2)} M ₽")
    print()

Для стоимости бага, равной 20 M ₽, стоит выбрать кр-ий A
Его стоимость составляет 0.12 M ₽

Для стоимости бага, равной 3 M ₽, стоит выбрать кр-ий A
Его стоимость составляет 0.04 M ₽

Для стоимости бага, равной 300 M ₽, стоит выбрать кр-ий C
Его стоимость составляет 1.01 M ₽



**Важно! Для остальных стратегий предполагается, что критерии слабо зависимы между собой, то есть плотность совместного распределение статистик у критериев близко к произведению плотностей**

<u> Обозначения</u>:  

$A$ - область кр-ия $A$  
$B$ - область кр-ия $B$  
$C$ - область кр-ия $C$  

### 2. Стратегия = принцип большинства

 Стратегия заключается в том, что мы принимаем гипотезу $H_1$ о том, что баг есть, если хотя бы 2 критерия отвергли гипотезу $H_0$

$\widehat{\alpha} = P\bigl( \text{сказали, что баг есть} | \text{бага нет} \bigr) = P\bigl(AB \cup BC \cup AC | \text{бага нет} \bigr) = 
\alpha_1 \alpha_2 + \alpha_2 \alpha_3 + \alpha_1 \alpha_3 - 2 \alpha_1 \alpha_2 \alpha_3$  
$\widehat{\beta} = P\bigl( \text{сказали, что бага нету} | \text{баг есть} \bigr) = \beta_1 \beta_2 + \beta_2 \beta_3 + \beta_1 \beta_3 - 2 \beta_1 \beta_2 \beta_3$

In [32]:
alpha_1, alpha_2, alpha_3 = 0.02, 0.05, 0.1
beta_1, beta_2, beta_3 = 0.5, 0.4, 0.3

alpha_hat = alpha_1 * alpha_2 + alpha_1 * alpha_3 + alpha_2 * alpha_3 - \
    2 * alpha_1 * alpha_2 * alpha_3
beta_hat = beta_1 * beta_2 + beta_1 * beta_3 + beta_2 * beta_3 - \
    2 * beta_1 * beta_2 * beta_3

print(f"alpha_hat = {alpha_hat}")
print(f"beta_hat = {beta_hat}")

for s2 in s2_arr:
    cr_voting_cost = criterion_cost(alpha_hat, beta_hat, p_bug, s1, s2)
    print(f"Для стоимости бага, равной {s2} M ₽, затраты данной стратегии", end=' ')
    print(f"составляют {round(cr_voting_cost, 2)} M ₽")

alpha_hat = 0.0078000000000000005
beta_hat = 0.35
Для стоимости бага, равной 20 M ₽, затраты данной стратегии составляют 0.08 M ₽
Для стоимости бага, равной 3 M ₽, затраты данной стратегии составляют 0.02 M ₽
Для стоимости бага, равной 300 M ₽, затраты данной стратегии составляют 1.06 M ₽


### 3. Стратегия = закон Мёрфи

Стратегия заключается в том, что мы принимаем гипотезу $H_1$ о том, что баг есть, если хотя бы 1 критерий отверг гипотезу $H_0$

$\widehat{\alpha} = P\bigl( \text{сказали, что баг есть} | \text{бага нет} \bigr) = P\bigl(A \cup B \cup C | \text{бага нет} \bigr) = \alpha_1 + \alpha_2 + \alpha_3 -
\alpha_1 \alpha_2 - \alpha_2 \alpha_3 - \alpha_1 \alpha_3 + \alpha_1 \alpha_2 \alpha_3$  
$\widehat{\beta} = P\bigl( \text{сказали, что бага нету} | \text{баг есть} \bigr) = P\bigl(\overline{A} \cap \overline{B} \cap \overline{C} | \text{бага нет} \bigr) = \beta_1 \beta_2 \beta_3$

In [33]:
alpha_hat = alpha_1 + alpha_2 + alpha_3 - \
    alpha_1 * alpha_2 - alpha_1 * alpha_3 - alpha_2 * alpha_3 + \
    alpha_1 * alpha_2 * alpha_3
beta_hat = beta_1 * beta_2 * beta_3

print(f"alpha_hat = {alpha_hat}")
print(f"beta_hat = {beta_hat}")

for s2 in s2_arr:
    cr_voting_cost = criterion_cost(alpha_hat, beta_hat, p_bug, s1, s2)
    print(f"Для стоимости бага, равной {s2} M ₽, затраты данной стратегии", end=' ')
    print(f"составляют {round(cr_voting_cost, 2)} M ₽")

alpha_hat = 0.1621
beta_hat = 0.06
Для стоимости бага, равной 20 M ₽, затраты данной стратегии составляют 0.18 M ₽
Для стоимости бага, равной 3 M ₽, затраты данной стратегии составляют 0.17 M ₽
Для стоимости бага, равной 300 M ₽, затраты данной стратегии составляют 0.35 M ₽


### 4. Стратегия = принцип ленивца

Стратегия заключается в том, что мы принимаем гипотезу $H_1$ о том, что баг есть, если все 3 критерия отвергли гипотезу $H_0$

$\widehat{\alpha} = P\bigl( \text{сказали, что баг есть} | \text{бага нет} \bigr) = P\bigl(A \cap B \cap C | \text{бага нет} \bigr) = \alpha_1 \alpha_2 \alpha_3$  
$\widehat{\beta} = P\bigl( \text{сказали, что бага нету} | \text{баг есть} \bigr) = P\bigl(\overline{A} \cup \overline{B} \cup \overline{C} | \text{бага нет} \bigr) = \beta_1 + \beta_2 + \beta_3 -
\beta_1 \beta_2 - \beta_2 \beta_3 - \beta_1 \beta_3 + \beta_1 \beta_2 \beta_3$

In [34]:
alpha_hat = alpha_1 * alpha_2 * alpha_3
beta_hat = beta_1 + beta_2 + beta_3 - \
    beta_1 * beta_2 - beta_1 * beta_3 - beta_2 * beta_3 + \
    beta_1 * beta_2 * beta_3

print(f"alpha_hat = {alpha_hat}")
print(f"beta_hat = {beta_hat}")

for s2 in s2_arr:
    cr_voting_cost = criterion_cost(alpha_hat, beta_hat, p_bug, s1, s2)
    print(f"Для стоимости бага, равной {s2} M ₽, затраты данной стратегии", end=' ')
    print(f"составляют {round(cr_voting_cost, 2)} M ₽")

alpha_hat = 0.0001
beta_hat = 0.79
Для стоимости бага, равной 20 M ₽, затраты данной стратегии составляют 0.16 M ₽
Для стоимости бага, равной 3 M ₽, затраты данной стратегии составляют 0.03 M ₽
Для стоимости бага, равной 300 M ₽, затраты данной стратегии составляют 2.37 M ₽


**Вывод:**  
Для стоимости бага, равной 20 M ₽, стоит выбрать стратегию "Принцип большинства"  
Его стоимость составляет 0.08 M ₽  

Для стоимости бага, равной 3 M ₽, стоит выбрать стратегию "Принцип большинства"  
Его стоимость составляет 0.02 M ₽  

Для стоимости бага, равной 300 M ₽, стоит выбрать стратегию "Закон Мёрфи"  
Его стоимость составляет 0.35 M ₽  